In [1]:
import yfinance as yf

# Function to get stock data
def get_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    return stock.history(start=start_date, end=end_date)


In [2]:
import requests

def fetch_news(query, start_date, end_date, api_key):
    url = f"https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "from": start_date,
        "to": end_date,
        "sortBy": "relevancy",
        "apiKey": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["articles"]
    else:
        raise Exception(f"Error fetching news: {response.status_code}")


In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    if scores["compound"] > 0.05:
        return "Positive"
    elif scores["compound"] < -0.05:
        return "Negative"
    else:
        return "Neutral"


In [4]:
def analyze_stock_sentiment(ticker, start_date, end_date, api_key):
    # Fetch stock data
    stock_data = get_stock_data(ticker, start_date, end_date)

    # Fetch news articles
    news_articles = fetch_news(ticker, start_date, end_date, api_key)

    # Perform sentiment analysis
    analyzed_news = []
    for article in news_articles:
        sentiment = analyze_sentiment(article["title"] + " " + article["description"])
        analyzed_news.append({
            "title": article["title"],
            "description": article["description"],
            "sentiment": sentiment,
        })

    return stock_data, analyzed_news


In [5]:
import matplotlib.pyplot as plt

# Visualize stock prices
def plot_stock_prices(stock_data):
    stock_data['Close'].plot(title="Stock Prices")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.show()
